In [0]:
#noetbook utilizado para a aula sobre SVM

In [0]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark


In [0]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("SVM_MLP").getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [0]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/iris_bezdekIris.csv,iris_bezdekIris.csv,4551
dbfs:/FileStore/tables/regressaoLinear-1.csv,regressaoLinear-1.csv,564
dbfs:/FileStore/tables/regressaoLinear.csv,regressaoLinear.csv,564


In [0]:
diretorioArvore="/FileStore/tables/iris_bezdekIris.csv"  #diretório que contém o arquivo a ser utilizado

In [0]:
#lendo arquivos armazenados CSV com o esquema definido
df_iris = spark.read.format('csv').options(inferSchema=True,header='false',delimiter=',').load(diretorioArvore)

In [0]:
df_iris.printSchema()

root
-- _c0: double (nullable = true)
-- _c1: double (nullable = true)
-- _c2: double (nullable = true)
-- _c3: double (nullable = true)
-- _c4: string (nullable = true)

In [0]:
#modificando o nome das colunas existentes no cabeçalho 
df_iris = df_iris.selectExpr("_c0 as sep_len", "_c1 as sep_wid", "_c2 as pet_len", "_c3 as pet_wid", "_c4 as label")

In [0]:
df_iris.show(5)

+-------+-------+-------+-------+-----------+
sep_len|sep_wid|pet_len|pet_wid| label|
+-------+-------+-------+-------+-----------+
 5.1| 3.5| 1.4| 0.2|Iris-setosa|
 4.9| 3.0| 1.4| 0.2|Iris-setosa|
 4.7| 3.2| 1.3| 0.2|Iris-setosa|
 4.6| 3.1| 1.5| 0.2|Iris-setosa|
 5.0| 3.6| 1.4| 0.2|Iris-setosa|
+-------+-------+-------+-------+-----------+
only showing top 5 rows

Conhecendo o Banco de Dados

In [0]:
#encontrando as "estatísticas"
df_iris.describe(['sep_len','sep_wid','pet_len','pet_wid']).show()

+-------+------------------+-------------------+------------------+------------------+
summary| sep_len| sep_wid| pet_len| pet_wid|
+-------+------------------+-------------------+------------------+------------------+
 count| 150| 150| 150| 150|
 mean| 5.843333333333335| 3.057333333333334|3.7580000000000027| 1.199333333333334|
 stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|
 min| 4.3| 2.0| 1.0| 0.1|
 max| 7.9| 4.4| 6.9| 2.5|
+-------+------------------+-------------------+------------------+------------------+

In [0]:
#definindo a visão do dataframe para ser utilizado como uma tabela pelo SQL
df_iris.createOrReplaceTempView("irisTable")

In [0]:
display(spark.sql('select * from irisTable '))

sep_len,sep_wid,pet_len,pet_wid,label
5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
5.0,3.6,1.4,0.2,Iris-setosa
5.4,3.9,1.7,0.4,Iris-setosa
4.6,3.4,1.4,0.3,Iris-setosa
5.0,3.4,1.5,0.2,Iris-setosa
4.4,2.9,1.4,0.2,Iris-setosa
4.9,3.1,1.5,0.1,Iris-setosa


Iniciando o Processo de Construção e Aplicação do SVM

In [0]:
from pyspark.ml.linalg import Vectors  #biblioteca que contém funções de Algebra Linear
from pyspark.ml.feature import VectorAssembler #biblioteca que contém as funções para a construção de vetores

In [0]:
#criando o vetor de características
vector_assembler = VectorAssembler(inputCols=["sep_len", "sep_wid", "pet_len", "pet_wid"],outputCol="features")
df_temp = vector_assembler.transform(df_iris)
df_temp.show(5)

+-------+-------+-------+-------+-----------+-----------------+
sep_len|sep_wid|pet_len|pet_wid| label| features|
+-------+-------+-------+-------+-----------+-----------------+
 5.1| 3.5| 1.4| 0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
 4.9| 3.0| 1.4| 0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
 4.7| 3.2| 1.3| 0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
 4.6| 3.1| 1.5| 0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
 5.0| 3.6| 1.4| 0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
+-------+-------+-------+-------+-----------+-----------------+
only showing top 5 rows

In [0]:
df_temp.printSchema()

root
-- sep_len: double (nullable = true)
-- sep_wid: double (nullable = true)
-- pet_len: double (nullable = true)
-- pet_wid: double (nullable = true)
-- label: string (nullable = true)
-- features: vector (nullable = true)

In [0]:
#removendo as colunas que não serão utilizadas
df_menor = df_temp.drop('sep_len', 'sep_wid', 'pet_len', 'pet_wid')
df_menor.show(5)

+-----------+-----------------+
 label| features|
+-----------+-----------------+
Iris-setosa|[5.1,3.5,1.4,0.2]|
Iris-setosa|[4.9,3.0,1.4,0.2]|
Iris-setosa|[4.7,3.2,1.3,0.2]|
Iris-setosa|[4.6,3.1,1.5,0.2]|
Iris-setosa|[5.0,3.6,1.4,0.2]|
+-----------+-----------------+
only showing top 5 rows

In [0]:
#aplicando as transformações para a coluna label
from pyspark.ml.feature import StringIndexer  #cria o 'vetor' para cada uma das classes existentes na coluna label

l_indexer = StringIndexer(inputCol="label", outputCol="labelEncoder")  #cria o objeto para a codificação
df_final = l_indexer.fit(df_menor).transform(df_menor)  #aplica a transformação

In [0]:
df_final.show(5)

+-----------+-----------------+------------+
 label| features|labelEncoder|
+-----------+-----------------+------------+
Iris-setosa|[5.1,3.5,1.4,0.2]| 0.0|
Iris-setosa|[4.9,3.0,1.4,0.2]| 0.0|
Iris-setosa|[4.7,3.2,1.3,0.2]| 0.0|
Iris-setosa|[4.6,3.1,1.5,0.2]| 0.0|
Iris-setosa|[5.0,3.6,1.4,0.2]| 0.0|
+-----------+-----------------+------------+
only showing top 5 rows

In [0]:
df_final.printSchema()

root
-- label: string (nullable = true)
-- features: vector (nullable = true)
-- labelEncoder: double (nullable = false)

Normalizando os dados

In [0]:
#normalizando os dados
from pyspark.ml.feature import MinMaxScaler  #biblioteca para colocar os valores entre 0 e 1
scaler = MinMaxScaler( inputCol="features", outputCol="scaledFeatures")  #criando o objeto para a escala
scalerModel = scaler.fit(df_final)

In [0]:
df_final = scalerModel.transform(df_final).drop('features').withColumnRenamed('scaledFeatures', 'features')

In [0]:
df_final.show()

+-----------+------------+--------------------+
 label|labelEncoder| features|
+-----------+------------+--------------------+
Iris-setosa| 0.0|[0.22222222222222...|
Iris-setosa| 0.0|[0.16666666666666...|
Iris-setosa| 0.0|[0.11111111111111...|
Iris-setosa| 0.0|[0.08333333333333...|
Iris-setosa| 0.0|[0.19444444444444...|
Iris-setosa| 0.0|[0.30555555555555...|
Iris-setosa| 0.0|[0.08333333333333...|
Iris-setosa| 0.0|[0.19444444444444...|
Iris-setosa| 0.0|[0.02777777777777...|
Iris-setosa| 0.0|[0.16666666666666...|
Iris-setosa| 0.0|[0.30555555555555...|
Iris-setosa| 0.0|[0.13888888888888...|
Iris-setosa| 0.0|[0.13888888888888...|
Iris-setosa| 0.0|[0.0,0.4166666666...|
Iris-setosa| 0.0|[0.41666666666666...|
Iris-setosa| 0.0|[0.38888888888888...|
Iris-setosa| 0.0|[0.30555555555555...|
Iris-setosa| 0.0|[0.22222222222222...|
Iris-setosa| 0.0|[0.38888888888888...|
Iris-setosa| 0.0|[0.22222222222222...|
+-----------+------------+--------------------+
only showing top 20 rows

Modificando o Dataset Para a Classifcação Binária

In [0]:
import pyspark.sql.functions as F
df_SVM=df_final.where((F.col("labelEncoder") == 0) | (F.col("labelEncoder") == 1))  #Transforma o dataset em um problema de classificação binária

In [0]:
df_SVM.show(150)

+---------------+------------+--------------------+
 label|labelEncoder| features|
+---------------+------------+--------------------+
 Iris-setosa| 0.0|[0.22222222222222...|
 Iris-setosa| 0.0|[0.16666666666666...|
 Iris-setosa| 0.0|[0.11111111111111...|
 Iris-setosa| 0.0|[0.08333333333333...|
 Iris-setosa| 0.0|[0.19444444444444...|
 Iris-setosa| 0.0|[0.30555555555555...|
 Iris-setosa| 0.0|[0.08333333333333...|
 Iris-setosa| 0.0|[0.19444444444444...|
 Iris-setosa| 0.0|[0.02777777777777...|
 Iris-setosa| 0.0|[0.16666666666666...|
 Iris-setosa| 0.0|[0.30555555555555...|
 Iris-setosa| 0.0|[0.13888888888888...|
 Iris-setosa| 0.0|[0.13888888888888...|
 Iris-setosa| 0.0|[0.0,0.4166666666...|
 Iris-setosa| 0.0|[0.41666666666666...|
 Iris-setosa| 0.0|[0.38888888888888...|
 Iris-setosa| 0.0|[0.30555555555555...|
 Iris-setosa| 0.0|[0.22222222222222...|
 Iris-setosa| 0.0|[0.38888888888888...|
 Iris-setosa| 0.0|[0.22222222222222...|
 Iris-setosa| 0.0|[0.30555555555555...|
 Iris-setosa| 0.0|[0.22222222222222...|
 Iris-setosa| 0.0|[0.08333333333333...|
 Iris-setosa| 0.0|[0.22222222222222...|
 Iris-setosa| 0.0|[0.13888888888888...|
 Iris-setosa| 0.0|[0.19444444444444...|
 Iris-setosa| 0.0|[0.19444444444444...|
 Iris-setosa| 0.0|[0.25000000000000...|
 Iris-setosa| 0.0|[0.25000000000000...|
 Iris-setosa| 0.0|[0.11111111111111...|
 Iris-setosa| 0.0|[0.13888888888888...|
 Iris-setosa| 0.0|[0.30555555555555...|
 Iris-setosa| 0.0|[0.25000000000000...|
 Iris-setosa| 0.0|[0.33333333333333...|
 Iris-setosa| 0.0|[0.16666666666666...|
 Iris-setosa| 0.0|[0.19444444444444...|
 Iris-setosa| 0.0|[0.33333333333333...|
 Iris-setosa| 0.0|[0.16666666666666...|
 Iris-setosa| 0.0|[0.02777777777777...|
 Iris-setosa| 0.0|[0.22222222222222...|
 Iris-setosa| 0.0|[0.19444444444444...|
 Iris-setosa| 0.0|[0.05555555555555...|
 Iris-setosa| 0.0|[0.02777777777777...|
 Iris-setosa| 0.0|[0.19444444444444...|
 Iris-setosa| 0.0|[0.22222222222222...|
 Iris-setosa| 0.0|[0.13888888888888...|
 Iris-setosa| 0.0|[0.22222222222222...|
 Iris-setosa| 0.0|[0.08333333333333...|
 Iris-setosa| 0.0|[0.27777777777777...|
 Iris-setosa| 0.0|[0.19444444444444...|
Iris-versicolor| 1.0|[0.74999999999999...|
Iris-versicolor| 1.0|[0.58333333333333...|
Iris-versicolor| 1.0|[0.72222222222222...|
Iris-versicolor| 1.0|[0.33333333333333...|
Iris-versicolor| 1.0|[0.61111111111111...|
Iris-versicolor| 1.0|[0.38888888888888...|
Iris-versicolor| 1.0|[0.55555555555555...|
Iris-versicolor| 1.0|[0.16666666666666...|
Iris-versicolor| 1.0|[0.63888888888888...|
Iris-versicolor| 1.0|[0.25000000000000...|
Iris-versicolor| 1.0|[0.19444444444444...|
Iris-versicolor| 1.0|[0.44444444444444...|
Iris-versicolor| 1.0|[0.47222222222222...|
Iris-versicolor| 1.0|[0.49999999999999...|
Iris-versicolor| 1.0|[0.36111111111111...|
Iris-versicolor| 1.0|[0.66666666666666...|
Iris-versicolor| 1.0|[0.36111111111111...|
Iris-versicolor| 1.0|[0.41666666666666...|
Iris-versicolor| 1.0|[0.52777777777777...|
Iris-versicolor| 1.0|[0.36111111111111...|
Iris-versicolor| 1.0|[0.44444444444444...|
Iris-versicolor| 1.0|[0.49999999999999...|
Iris-versicolor| 1.0|[0.55555555555555...|
Iris-versicolor| 1.0|[0.49999999999999...|
Iris-versicolor| 1.0|[0.58333333333333...|
Iris-versicolor| 1.0|[0.63888888888888...|
Iris-versicolor| 1.0|[0.69444444444444...|
Iris-versicolor| 1.0|[0.66666666666666...|
Iris-versicolor| 1.0|[0.47222222222222...|
Iris-versicolor| 1.0|[0.38888888888888...|
Iris-versicolor| 1.0|[0.33333333333333...|
Iris-versicolor| 1.0|[0.33333333333333...|
Iris-versicolor| 1.0|[0.41666666666666...|
Iris-versicolor| 1.0|[0.47222222222222...|
Iris-versicolor| 1.0|[0.30555555555555...|
Iris-versicolor| 1.0|[0.47222222222222...|
Iris-versicolor| 1.0|[0.66666666666666...|
Iris-versicolor| 1.0|[0.55555555555555...|
Iris-versicolor| 1.0|[0.36111111111111...|
Iris-versicolor| 1.0|[0.33333333333333...|
Iris-versicolor| 1.0|[0.33333333333333...|
Iris-versicolor| 1.0|[0.49999999999999...|
Iris-versicolor| 1.0|[0.41666666666666...|
Iris-versicolor|

In [0]:
#removendo as colunas que não serão utilizadas
df_SVM = df_SVM.drop('label')
df_SVM.show(130)

+------------+--------------------+
labelEncoder| features|
+------------+--------------------+
 0.0|[0.22222222222222...|
 0.0|[0.16666666666666...|
 0.0|[0.11111111111111...|
 0.0|[0.08333333333333...|
 0.0|[0.19444444444444...|
 0.0|[0.30555555555555...|
 0.0|[0.08333333333333...|
 0.0|[0.19444444444444...|
 0.0|[0.02777777777777...|
 0.0|[0.16666666666666...|
 0.0|[0.30555555555555...|
 0.0|[0.13888888888888...|
 0.0|[0.13888888888888...|
 0.0|[0.0,0.4166666666...|
 0.0|[0.41666666666666...|
 0.0|[0.38888888888888...|
 0.0|[0.30555555555555...|
 0.0|[0.22222222222222...|
 0.0|[0.38888888888888...|
 0.0|[0.22222222222222...|
 0.0|[0.30555555555555...|
 0.0|[0.22222222222222...|
 0.0|[0.08333333333333...|
 0.0|[0.22222222222222...|
 0.0|[0.13888888888888...|
 0.0|[0.19444444444444...|
 0.0|[0.19444444444444...|
 0.0|[0.25000000000000...|
 0.0|[0.25000000000000...|
 0.0|[0.11111111111111...|
 0.0|[0.13888888888888...|
 0.0|[0.30555555555555...|
 0.0|[0.25000000000000...|
 0.0|[0.33333333333333...|
 0.0|[0.16666666666666...|
 0.0|[0.19444444444444...|
 0.0|[0.33333333333333...|
 0.0|[0.16666666666666...|
 0.0|[0.02777777777777...|
 0.0|[0.22222222222222...|
 0.0|[0.19444444444444...|
 0.0|[0.05555555555555...|
 0.0|[0.02777777777777...|
 0.0|[0.19444444444444...|
 0.0|[0.22222222222222...|
 0.0|[0.13888888888888...|
 0.0|[0.22222222222222...|
 0.0|[0.08333333333333...|
 0.0|[0.27777777777777...|
 0.0|[0.19444444444444...|
 1.0|[0.74999999999999...|
 1.0|[0.58333333333333...|
 1.0|[0.72222222222222...|
 1.0|[0.33333333333333...|
 1.0|[0.61111111111111...|
 1.0|[0.38888888888888...|
 1.0|[0.55555555555555...|
 1.0|[0.16666666666666...|
 1.0|[0.63888888888888...|
 1.0|[0.25000000000000...|
 1.0|[0.19444444444444...|
 1.0|[0.44444444444444...|
 1.0|[0.47222222222222...|
 1.0|[0.49999999999999...|
 1.0|[0.36111111111111...|
 1.0|[0.66666666666666...|
 1.0|[0.36111111111111...|
 1.0|[0.41666666666666...|
 1.0|[0.52777777777777...|
 1.0|[0.36111111111111...|
 1.0|[0.44444444444444...|
 1.0|[0.49999999999999...|
 1.0|[0.55555555555555...|
 1.0|[0.49999999999999...|
 1.0|[0.58333333333333...|
 1.0|[0.63888888888888...|
 1.0|[0.69444444444444...|
 1.0|[0.66666666666666...|
 1.0|[0.47222222222222...|
 1.0|[0.38888888888888...|
 1.0|[0.33333333333333...|
 1.0|[0.33333333333333...|
 1.0|[0.41666666666666...|
 1.0|[0.47222222222222...|
 1.0|[0.30555555555555...|
 1.0|[0.47222222222222...|
 1.0|[0.66666666666666...|
 1.0|[0.55555555555555...|
 1.0|[0.36111111111111...|
 1.0|[0.33333333333333...|
 1.0|[0.33333333333333...|
 1.0|[0.49999999999999...|
 1.0|[0.41666666666666...|
 1.0|[0.19444444444444...|
 1.0|[0.36111111111111...|
 1.0|[0.38888888888888...|
 1.0|[0.38888888888888...|
 1.0|[0.52777777777777...|
 1.0|[0.22222222222222...|
 1.0|[0.38888888888888...|
+------------+--------------------+

In [0]:
df_SVM=df_SVM.selectExpr('features',"labelEncoder as label")

In [0]:
df_SVM.show(5)

+--------------------+-----+
 features|label|
+--------------------+-----+
[0.22222222222222...| 0.0|
[0.16666666666666...| 0.0|
[0.11111111111111...| 0.0|
[0.08333333333333...| 0.0|
[0.19444444444444...| 0.0|
+--------------------+-----+
only showing top 5 rows

Dividindo o dataset entre treinamento e teste

In [0]:
#dividindo entre dados de treinamento e teste
(train, test) = df_SVM.randomSplit([0.7, 0.3])

In [0]:
train.show(100)

+--------------------+-----+
 features|label|
+--------------------+-----+
[0.0,0.4166666666...| 0.0|
[0.02777777777777...| 0.0|
[0.08333333333333...| 0.0|
[0.11111111111111...| 0.0|
[0.11111111111111...| 0.0|
[0.13888888888888...| 0.0|
[0.13888888888888...| 0.0|
[0.13888888888888...| 0.0|
[0.13888888888888...| 0.0|
[0.16666666666666...| 1.0|
[0.16666666666666...| 0.0|
[0.16666666666666...| 0.0|
[0.16666666666666...| 0.0|
[0.19444444444444...| 0.0|
[0.19444444444444...| 0.0|
[0.19444444444444...| 0.0|
[0.19444444444444...| 0.0|
[0.19444444444444...| 0.0|
[0.19444444444444...| 0.0|
[0.19444444444444...| 0.0|
[0.22222222222222...| 1.0|
[0.22222222222222...| 0.0|
[0.22222222222222...| 0.0|
[0.22222222222222...| 0.0|
[0.22222222222222...| 0.0|
[0.22222222222222...| 0.0|
[0.22222222222222...| 0.0|
[0.22222222222222...| 0.0|
[0.25000000000000...| 1.0|
[0.25000000000000...| 0.0|
[0.25000000000000...| 0.0|
[0.25000000000000...| 0.0|
[0.27777777777777...| 0.0|
[0.30555555555555...| 1.0|
[0.30555555555555...| 0.0|
[0.30555555555555...| 0.0|
[0.30555555555555...| 0.0|
[0.30555555555555...| 0.0|
[0.30555555555555...| 0.0|
[0.33333333333333...| 1.0|
[0.33333333333333...| 1.0|
[0.33333333333333...| 1.0|
[0.33333333333333...| 1.0|
[0.33333333333333...| 0.0|
[0.36111111111111...| 1.0|
[0.36111111111111...| 1.0|
[0.36111111111111...| 1.0|
[0.36111111111111...| 1.0|
[0.38888888888888...| 1.0|
[0.38888888888888...| 1.0|
[0.38888888888888...| 0.0|
[0.41666666666666...| 1.0|
[0.41666666666666...| 1.0|
[0.44444444444444...| 1.0|
[0.44444444444444...| 1.0|
[0.47222222222222...| 1.0|
[0.47222222222222...| 1.0|
[0.49999999999999...| 1.0|
[0.49999999999999...| 1.0|
[0.49999999999999...| 1.0|
[0.49999999999999...| 1.0|
[0.52777777777777...| 1.0|
[0.52777777777777...| 1.0|
[0.55555555555555...| 1.0|
[0.58333333333333...| 1.0|
[0.58333333333333...| 1.0|
[0.63888888888888...| 1.0|
[0.66666666666666...| 1.0|
[0.66666666666666...| 1.0|
[0.69444444444444...| 1.0|
[0.72222222222222...| 1.0|
[0.74999999999999...| 1.0|
+--------------------+-----+

In [0]:
print("Dados para treinamento: ", train.count())
print("Dados para teste: ", test.count())

Dados para treinamento: 72
Dados para teste: 28

Definindo o Algoritmo

In [0]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel  #modelo de svm
from pyspark.ml.evaluation import MulticlassClassificationEvaluator  #utilizada para encontrar as métricas de desempenho
from pyspark.mllib.linalg import Vectors  #vetores densos

In [0]:
from pyspark.mllib.util import MLUtils
df_train = MLUtils.convertVectorColumnsFromML(train, "features")
df_test = MLUtils.convertVectorColumnsFromML(test, "features")

In [0]:
df_train.show(5,False)

+----------------------------------------------------------------------------------+-----+
features |label|
+----------------------------------------------------------------------------------+-----+
[0.0,0.41666666666666663,0.016949152542372895,0.0] |0.0 |
[0.027777777777777922,0.37499999999999994,0.06779661016949151,0.04166666666666667]|0.0 |
[0.08333333333333327,0.6666666666666666,0.0,0.04166666666666667] |0.0 |
[0.11111111111111119,0.5,0.05084745762711865,0.04166666666666667] |0.0 |
[0.11111111111111119,0.5,0.1016949152542373,0.04166666666666667] |0.0 |
+----------------------------------------------------------------------------------+-----+
only showing top 5 rows

In [0]:
from pyspark.mllib.regression import LabeledPoint  #cria a "linha" (características e label) a ser utilizada

trainingData = df_train.rdd.map(lambda row:LabeledPoint(row.label,row.features))  #aplica o label ao treinamento
testingData = df_test.rdd.map(lambda row:LabeledPoint(row.label,row.features))  #aplica o label ao teste

In [0]:
for xs in trainingData.take(10):
        print(xs)

(0.0,[0.0,0.41666666666666663,0.016949152542372895,0.0])
(0.0,[0.027777777777777922,0.37499999999999994,0.06779661016949151,0.04166666666666667])
(0.0,[0.08333333333333327,0.6666666666666666,0.0,0.04166666666666667])
(0.0,[0.11111111111111119,0.5,0.05084745762711865,0.04166666666666667])
(0.0,[0.11111111111111119,0.5,0.1016949152542373,0.04166666666666667])
(0.0,[0.13888888888888887,0.41666666666666663,0.06779661016949151,0.0])
(0.0,[0.13888888888888887,0.4583333333333333,0.1016949152542373,0.04166666666666667])
(0.0,[0.13888888888888887,0.5833333333333333,0.1016949152542373,0.04166666666666667])
(0.0,[0.13888888888888887,0.5833333333333333,0.15254237288135591,0.04166666666666667])
(1.0,[0.1666666666666668,0.1666666666666666,0.38983050847457623,0.375])

In [0]:
#contrução do modelo
modelSVM = SVMWithSGD.train(trainingData, iterations=100)

In [0]:
#realizando a previsão
labelsAndPreds = testingData.map(lambda p: (p.label, modelSVM.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(testingData.count())
print("Erro na previsão: ",trainErr)

Erro na previsão: 0.0